# Setup
(No need to read)

In [6]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    # !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    # %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-qip1qvb5
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-qip1qvb5
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit fa287750606075574df2c538058e67d648e2f952
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached accelerate-0.24.1-py3-none-any.whl (261 kB)
  Using cached beartype-0.14.1-py3-none-any.whl (739 kB)
  Using cached datasets-2.14.6-py3-none-any.whl (493 kB)
  Using cached einops-0.7.0-py3-none-any.whl (44 kB)
  Using cached fancy_einsum-0.0.3-py3-none-any.whl (6.2 kB)
  Using cached jaxtyping-0.2.23-py3-none-any.whl (29 kB)
  Using cached numpy-1.26.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Using cached nvidia_cublas_cu12-12.3.2.9-py3-none-man

In [7]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [8]:
# import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

In [9]:
model = HookedTransformer.from_pretrained("gpt2-small")

Loaded pretrained model gpt2-small into HookedTransformer


# len 2

In [ ]:
example_prompt = "3 4"
example_answer = " 5"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

In [ ]:
example_prompt = "33 34"
example_answer = " 35"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

In [ ]:
example_prompt = "null 33 34"
example_answer = " 35"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

In [ ]:
example_prompt = "32 33 34"
example_answer = " 35"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

In [ ]:
example_prompt = "1 2"
example_answer = " 3"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

In [ ]:
example_prompt = "March April"
example_answer = " May"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

# random

In [ ]:
#@title prompt_dict

prompt_dict = [{'S1': '20',
  'S2': '22',
  'S3': '26',
  'S4': '25',
  'corr': '20',
  'incorr': '5',
  'text': '20 22 26 25'},
 {'S1': '29',
  'S2': '16',
  'S3': '17',
  'S4': '20',
  'corr': '29',
  'incorr': '6',
  'text': '29 16 17 20'},
 {'S1': '20',
  'S2': '20',
  'S3': '19',
  'S4': '29',
  'corr': '20',
  'incorr': '7',
  'text': '20 20 19 29'},
 {'S1': '16',
  'S2': '16',
  'S3': '20',
  'S4': '25',
  'corr': '16',
  'incorr': '8',
  'text': '16 16 20 25'},
 {'S1': '20',
  'S2': '25',
  'S3': '22',
  'S4': '30',
  'corr': '20',
  'incorr': '9',
  'text': '20 25 22 30'},
 {'S1': '22',
  'S2': '17',
  'S3': '22',
  'S4': '27',
  'corr': '22',
  'incorr': '10',
  'text': '22 17 22 27'},
 {'S1': '17',
  'S2': '15',
  'S3': '29',
  'S4': '22',
  'corr': '17',
  'incorr': '11',
  'text': '17 15 29 22'},
 {'S1': '20',
  'S2': '20',
  'S3': '23',
  'S4': '30',
  'corr': '20',
  'incorr': '12',
  'text': '20 20 23 30'},
 {'S1': '15',
  'S2': '28',
  'S3': '24',
  'S4': '25',
  'corr': '15',
  'incorr': '13',
  'text': '15 28 24 25'},
 {'S1': '18',
  'S2': '19',
  'S3': '15',
  'S4': '18',
  'corr': '18',
  'incorr': '14',
  'text': '18 19 15 18'}]

In [ ]:
example_prompt = "20 20 19 29"
example_answer = " 20"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

In [ ]:
for prompt_dict in prompt_dict:
    example_prompt = prompt_dict['text']
    example_answer = prompt_dict['corr']
    utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

# add 2

In [ ]:
example_prompt = "2 4 6 8 10 12 14 18"
example_answer = " 20"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

In [ ]:
example_prompt = "4 6 8 10 12 14 18 20 22"
example_answer = " 24"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

In [ ]:
example_prompt = "12 14 18 20 22"
example_answer = " 24"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

In [ ]:
example_prompt = "12 14 18 20 22 24 26 28 30"
example_answer = " 32"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

# arithm plus 1

In [ ]:
example_prompt = "1 + 1 ="
example_answer = " 2"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

In [ ]:
example_prompt = "10 + 1 ="
example_answer = " 11"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

# 1 table 2

In [10]:
example_prompt = "1 table 2. 10 table 11. 3 table 4. 5 table"
example_answer = " 6"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' table', ' 2', '.', ' 10', ' table', ' 11', '.', ' 3', ' table', ' 4', '.', ' 5', ' table']
Tokenized answer: [' 6']


Performance on answer token:
Rank: 1        Logit: 14.86 Prob: 19.11% Token: | 6|

Top 0th token. Logit: 15.55 Prob: 38.12% Token: | 5|
Top 1th token. Logit: 14.86 Prob: 19.11% Token: | 6|
Top 2th token. Logit: 14.16 Prob:  9.45% Token: | 7|
Top 3th token. Logit: 13.34 Prob:  4.19% Token: | 1|
Top 4th token. Logit: 13.33 Prob:  4.14% Token: | 11|
Top 5th token. Logit: 13.11 Prob:  3.31% Token: | 12|
Top 6th token. Logit: 12.92 Prob:  2.73% Token: |
|
Top 7th token. Logit: 12.10 Prob:  1.21% Token: | 13|
Top 8th token. Logit: 11.96 Prob:  1.05% Token: | 8|
Top 9th token. Logit: 11.87 Prob:  0.96% Token: | 4|


Ranks of the answer tokens: [(' 6', 1)]

In [11]:
example_prompt = "1 table 2. 10 table 11. 3 table 4. 15 table"
example_answer = " 16"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' table', ' 2', '.', ' 10', ' table', ' 11', '.', ' 3', ' table', ' 4', '.', ' 15', ' table']
Tokenized answer: [' 16']


Performance on answer token:
Rank: 15       Logit: 11.70 Prob:  0.68% Token: | 16|

Top 0th token. Logit: 16.09 Prob: 54.95% Token: | 5|
Top 1th token. Logit: 13.54 Prob:  4.29% Token: | 12|
Top 2th token. Logit: 13.53 Prob:  4.25% Token: | 6|
Top 3th token. Logit: 13.32 Prob:  3.43% Token: | 1|
Top 4th token. Logit: 13.28 Prob:  3.30% Token: |
|
Top 5th token. Logit: 13.09 Prob:  2.72% Token: | 7|
Top 6th token. Logit: 13.00 Prob:  2.49% Token: | 15|
Top 7th token. Logit: 12.97 Prob:  2.41% Token: | 13|
Top 8th token. Logit: 12.95 Prob:  2.37% Token: | 11|
Top 9th token. Logit: 12.80 Prob:  2.05% Token: | 17|


Ranks of the answer tokens: [(' 16', 15)]

In [12]:
example_prompt = "1 table 2. 10 table 11. 3 table 4. 101 table"
example_answer = " 102"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' table', ' 2', '.', ' 10', ' table', ' 11', '.', ' 3', ' table', ' 4', '.', ' 101', ' table']
Tokenized answer: [' 102']


Performance on answer token:
Rank: 48       Logit:  9.23 Prob:  0.08% Token: | 102|

Top 0th token. Logit: 15.14 Prob: 29.44% Token: | 12|
Top 1th token. Logit: 14.53 Prob: 15.94% Token: | 5|
Top 2th token. Logit: 14.01 Prob:  9.46% Token: | 13|
Top 3th token. Logit: 13.57 Prob:  6.13% Token: | 1|
Top 4th token. Logit: 13.53 Prob:  5.85% Token: | 11|
Top 5th token. Logit: 12.67 Prob:  2.47% Token: | 10|
Top 6th token. Logit: 12.65 Prob:  2.44% Token: |
|
Top 7th token. Logit: 12.36 Prob:  1.83% Token: | 2|
Top 8th token. Logit: 12.28 Prob:  1.68% Token: | Table|
Top 9th token. Logit: 12.05 Prob:  1.34% Token: | 15|


Ranks of the answer tokens: [(' 102', 48)]

# incr seq lens

In [14]:
example_prompt = "2"
example_answer = " 3"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '2']
Tokenized answer: [' 3']


Performance on answer token:
Rank: 33       Logit:  8.15 Prob:  0.24% Token: | 3|

Top 0th token. Logit: 12.87 Prob: 27.34% Token: |.|
Top 1th token. Logit: 11.11 Prob:  4.71% Token: |:|
Top 2th token. Logit: 10.79 Prob:  3.39% Token: |
|
Top 3th token. Logit: 10.61 Prob:  2.83% Token: |)|
Top 4th token. Logit: 10.58 Prob:  2.77% Token: |/|
Top 5th token. Logit: 10.39 Prob:  2.28% Token: |,|
Top 6th token. Logit: 10.34 Prob:  2.18% Token: |-|
Top 7th token. Logit: 10.26 Prob:  2.00% Token: |nd|
Top 8th token. Logit:  9.88 Prob:  1.37% Token: | of|
Top 9th token. Logit:  9.51 Prob:  0.94% Token: | hours|


Ranks of the answer tokens: [(' 3', 33)]

In [13]:
example_prompt = "1 1 1 2"
example_answer = " 3"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 1', ' 1', ' 2']
Tokenized answer: [' 3']


Performance on answer token:
Rank: 0        Logit: 16.37 Prob: 72.11% Token: | 3|

Top 0th token. Logit: 16.37 Prob: 72.11% Token: | 3|
Top 1th token. Logit: 13.63 Prob:  4.70% Token: | 1|
Top 2th token. Logit: 13.57 Prob:  4.38% Token: | 2|
Top 3th token. Logit: 13.06 Prob:  2.65% Token: | 4|
Top 4th token. Logit: 12.87 Prob:  2.19% Token: |
|
Top 5th token. Logit: 12.79 Prob:  2.02% Token: | 5|
Top 6th token. Logit: 11.79 Prob:  0.74% Token: |.|
Top 7th token. Logit: 11.53 Prob:  0.57% Token: | 0|
Top 8th token. Logit: 11.16 Prob:  0.40% Token: | 7|
Top 9th token. Logit: 11.00 Prob:  0.34% Token: | -|


Ranks of the answer tokens: [(' 3', 0)]

In [15]:
example_prompt = "1 2"
example_answer = " 3"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 2']
Tokenized answer: [' 3']


Performance on answer token:
Rank: 0        Logit: 16.69 Prob: 89.34% Token: | 3|

Top 0th token. Logit: 16.69 Prob: 89.34% Token: | 3|
Top 1th token. Logit: 12.56 Prob:  1.44% Token: |
|
Top 2th token. Logit: 11.64 Prob:  0.58% Token: | 4|
Top 3th token. Logit: 11.10 Prob:  0.33% Token: | 1|
Top 4th token. Logit: 10.85 Prob:  0.26% Token: | 5|
Top 5th token. Logit: 10.80 Prob:  0.25% Token: | Next|
Top 6th token. Logit: 10.78 Prob:  0.24% Token: |.|
Top 7th token. Logit: 10.35 Prob:  0.16% Token: | 2|
Top 8th token. Logit: 10.16 Prob:  0.13% Token: | /|
Top 9th token. Logit: 10.11 Prob:  0.12% Token: |/|


Ranks of the answer tokens: [(' 3', 0)]

In [19]:
example_prompt = "96"
example_answer = " 97"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '96']
Tokenized answer: [' 97']


Performance on answer token:
Rank: 518      Logit:  5.44 Prob:  0.01% Token: | 97|

Top 0th token. Logit: 12.10 Prob: 10.96% Token: |.|
Top 1th token. Logit: 12.08 Prob: 10.73% Token: |%|
Top 2th token. Logit: 11.57 Prob:  6.45% Token: |
|
Top 3th token. Logit: 11.17 Prob:  4.32% Token: | SHARES|
Top 4th token. Logit: 10.68 Prob:  2.63% Token: | %|
Top 5th token. Logit: 10.24 Prob:  1.71% Token: |-|
Top 6th token. Logit: 10.20 Prob:  1.63% Token: |1|
Top 7th token. Logit:  9.91 Prob:  1.23% Token: |2|
Top 8th token. Logit:  9.80 Prob:  1.10% Token: | votes|
Top 9th token. Logit:  9.78 Prob:  1.07% Token: |3|


Ranks of the answer tokens: [(' 97', 518)]

In [18]:
example_prompt = "95 96"
example_answer = " 97"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '95', ' 96']
Tokenized answer: [' 97']


Performance on answer token:
Rank: 0        Logit: 13.28 Prob: 11.88% Token: | 97|

Top 0th token. Logit: 13.28 Prob: 11.88% Token: | 97|
Top 1th token. Logit: 12.62 Prob:  6.15% Token: | 96|
Top 2th token. Logit: 12.35 Prob:  4.68% Token: |.|
Top 3th token. Logit: 12.19 Prob:  3.99% Token: |
|
Top 4th token. Logit: 12.00 Prob:  3.30% Token: | 98|
Top 5th token. Logit: 11.99 Prob:  3.29% Token: | 99|
Top 6th token. Logit: 11.64 Prob:  2.30% Token: | 95|
Top 7th token. Logit: 11.50 Prob:  2.00% Token: | 0|
Top 8th token. Logit: 11.36 Prob:  1.75% Token: | 100|
Top 9th token. Logit: 11.11 Prob:  1.35% Token: | 1|


Ranks of the answer tokens: [(' 97', 0)]

In [20]:
example_prompt = "94 95 96"
example_answer = " 97"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '94', ' 95', ' 96']
Tokenized answer: [' 97']


Performance on answer token:
Rank: 0        Logit: 18.81 Prob: 73.38% Token: | 97|

Top 0th token. Logit: 18.81 Prob: 73.38% Token: | 97|
Top 1th token. Logit: 15.94 Prob:  4.14% Token: | 99|
Top 2th token. Logit: 15.66 Prob:  3.13% Token: |
|
Top 3th token. Logit: 15.54 Prob:  2.78% Token: | 98|
Top 4th token. Logit: 15.41 Prob:  2.45% Token: | 95|
Top 5th token. Logit: 15.06 Prob:  1.72% Token: | 96|
Top 6th token. Logit: 14.56 Prob:  1.04% Token: | 93|
Top 7th token. Logit: 14.42 Prob:  0.91% Token: | 100|
Top 8th token. Logit: 14.05 Prob:  0.63% Token: | 107|
Top 9th token. Logit: 13.81 Prob:  0.49% Token: | 103|


Ranks of the answer tokens: [(' 97', 0)]

# born in

In [22]:
example_prompt = "Bob was born in 1990 and died in 19"
example_answer = " 99"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Bob', ' was', ' born', ' in', ' 1990', ' and', ' died', ' in', ' 19']
Tokenized answer: [' 99']


Performance on answer token:
Rank: 2232     Logit:  4.35 Prob:  0.00% Token: | 99|

Top 0th token. Logit: 14.97 Prob: 21.13% Token: | years|
Top 1th token. Logit: 14.75 Prob: 17.04% Token: | months|
Top 2th token. Logit: 13.84 Prob:  6.81% Token: | days|
Top 3th token. Logit: 13.22 Prob:  3.66% Token: | minutes|
Top 4th token. Logit: 13.08 Prob:  3.18% Token: |th|
Top 5th token. Logit: 12.78 Prob:  2.36% Token: | June|
Top 6th token. Logit: 12.46 Prob:  1.72% Token: | March|
Top 7th token. Logit: 12.37 Prob:  1.58% Token: | August|
Top 8th token. Logit: 12.30 Prob:  1.46% Token: | September|
Top 9th token. Logit: 12.23 Prob:  1.36% Token: | April|


Ranks of the answer tokens: [(' 99', 2232)]

In [25]:
example_prompt = "The war started in 1990 and ended in 19"
example_answer = " 99"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'The', ' war', ' started', ' in', ' 1990', ' and', ' ended', ' in', ' 19']
Tokenized answer: [' 99']


Performance on answer token:
Rank: 978      Logit:  6.19 Prob:  0.00% Token: | 99|

Top 0th token. Logit: 14.90 Prob:  9.42% Token: | months|
Top 1th token. Logit: 14.70 Prob:  7.70% Token: | September|
Top 2th token. Logit: 14.69 Prob:  7.59% Token: | March|
Top 3th token. Logit: 14.65 Prob:  7.35% Token: | August|
Top 4th token. Logit: 14.32 Prob:  5.28% Token: | February|
Top 5th token. Logit: 14.31 Prob:  5.24% Token: | July|
Top 6th token. Logit: 14.30 Prob:  5.18% Token: | November|
Top 7th token. Logit: 14.19 Prob:  4.64% Token: | June|
Top 8th token. Logit: 14.13 Prob:  4.34% Token: | April|
Top 9th token. Logit: 14.11 Prob:  4.29% Token: | years|


Ranks of the answer tokens: [(' 99', 978)]

In [28]:
example_prompt = "The war started in 1790 and ended in 17"
example_answer = "99"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'The', ' war', ' started', ' in', ' 17', '90', ' and', ' ended', ' in', ' 17']
Tokenized answer: [' 99']


Performance on answer token:
Rank: 233      Logit: 13.23 Prob:  0.00% Token: | 99|

Top 0th token. Logit: 26.85 Prob: 20.73% Token: |95|
Top 1th token. Logit: 26.25 Prob: 11.40% Token: |91|
Top 2th token. Logit: 26.13 Prob: 10.10% Token: |92|
Top 3th token. Logit: 26.10 Prob:  9.80% Token: |94|
Top 4th token. Logit: 25.94 Prob:  8.32% Token: |99|
Top 5th token. Logit: 25.70 Prob:  6.54% Token: |93|
Top 6th token. Logit: 25.69 Prob:  6.51% Token: |96|
Top 7th token. Logit: 25.63 Prob:  6.14% Token: |98|
Top 8th token. Logit: 25.60 Prob:  5.97% Token: |97|
Top 9th token. Logit: 25.43 Prob:  5.04% Token: |90|


Ranks of the answer tokens: [(' 99', 233)]

In [29]:
example_prompt = "Bob was born in 1790 and died in 17"
example_answer = " 99"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Bob', ' was', ' born', ' in', ' 17', '90', ' and', ' died', ' in', ' 17']
Tokenized answer: [' 99']


Performance on answer token:
Rank: 347      Logit: 12.54 Prob:  0.00% Token: | 99|

Top 0th token. Logit: 25.69 Prob: 11.15% Token: |95|
Top 1th token. Logit: 25.63 Prob: 10.47% Token: |94|
Top 2th token. Logit: 25.32 Prob:  7.69% Token: |92|
Top 3th token. Logit: 25.29 Prob:  7.46% Token: |91|
Top 4th token. Logit: 25.16 Prob:  6.61% Token: |99|
Top 5th token. Logit: 25.07 Prob:  5.99% Token: |90|
Top 6th token. Logit: 25.02 Prob:  5.73% Token: |97|
Top 7th token. Logit: 24.98 Prob:  5.49% Token: |98|
Top 8th token. Logit: 24.90 Prob:  5.07% Token: |93|
Top 9th token. Logit: 24.85 Prob:  4.84% Token: |96|


Ranks of the answer tokens: [(' 99', 347)]

In [30]:
example_prompt = "Bob died in 1790 and was born in 17"
example_answer = " 99"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Bob', ' died', ' in', ' 17', '90', ' and', ' was', ' born', ' in', ' 17']
Tokenized answer: [' 99']


Performance on answer token:
Rank: 865      Logit: 10.27 Prob:  0.00% Token: | 99|

Top 0th token. Logit: 23.94 Prob:  9.59% Token: |91|
Top 1th token. Logit: 23.82 Prob:  8.49% Token: |94|
Top 2th token. Logit: 23.73 Prob:  7.80% Token: |92|
Top 3th token. Logit: 23.50 Prob:  6.20% Token: |95|
Top 4th token. Logit: 23.46 Prob:  5.93% Token: |90|
Top 5th token. Logit: 23.25 Prob:  4.79% Token: |99|
Top 6th token. Logit: 23.21 Prob:  4.61% Token: |96|
Top 7th token. Logit: 23.12 Prob:  4.24% Token: |93|
Top 8th token. Logit: 22.98 Prob:  3.68% Token: |89|
Top 9th token. Logit: 22.97 Prob:  3.63% Token: |97|


Ranks of the answer tokens: [(' 99', 865)]

# medium less-than

In [31]:
model = HookedTransformer.from_pretrained("gpt2-medium")

Loaded pretrained model gpt2-medium into HookedTransformer


In [32]:
example_prompt = "The war ended in 1790 and started in 17"
example_answer = " 99"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'The', ' war', ' ended', ' in', ' 17', '90', ' and', ' started', ' in', ' 17']
Tokenized answer: [' 99']


Performance on answer token:
Rank: 275      Logit:  9.80 Prob:  0.00% Token: | 99|

Top 0th token. Logit: 22.54 Prob: 24.98% Token: |91|
Top 1th token. Logit: 21.80 Prob: 11.95% Token: |94|
Top 2th token. Logit: 21.69 Prob: 10.71% Token: |93|
Top 3th token. Logit: 21.54 Prob:  9.20% Token: |92|
Top 4th token. Logit: 21.32 Prob:  7.36% Token: |95|
Top 5th token. Logit: 21.11 Prob:  5.98% Token: |96|
Top 6th token. Logit: 20.94 Prob:  5.07% Token: |99|
Top 7th token. Logit: 20.61 Prob:  3.64% Token: |90|
Top 8th token. Logit: 20.59 Prob:  3.58% Token: |98|
Top 9th token. Logit: 20.13 Prob:  2.26% Token: |97|


Ranks of the answer tokens: [(' 99', 275)]

In [33]:
example_prompt = "The war ended in 1790 and started in 1780. The war ended in 1750 and started in 17"
example_answer = " 99"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'The', ' war', ' ended', ' in', ' 17', '90', ' and', ' started', ' in', ' 17', '80', '.', ' The', ' war', ' ended', ' in', ' 17', '50', ' and', ' started', ' in', ' 17']
Tokenized answer: [' 99']


Performance on answer token:
Rank: 1233     Logit:  8.79 Prob:  0.00% Token: | 99|

Top 0th token. Logit: 22.98 Prob:  6.84% Token: |50|
Top 1th token. Logit: 22.86 Prob:  6.02% Token: |40|
Top 2th token. Logit: 22.62 Prob:  4.76% Token: |60|
Top 3th token. Logit: 22.57 Prob:  4.51% Token: |30|
Top 4th token. Logit: 22.55 Prob:  4.41% Token: |80|
Top 5th token. Logit: 21.96 Prob:  2.46% Token: |70|
Top 6th token. Logit: 21.93 Prob:  2.39% Token: |90|
Top 7th token. Logit: 21.76 Prob:  2.02% Token: |20|
Top 8th token. Logit: 21.73 Prob:  1.95% Token: |55|
Top 9th token. Logit: 21.57 Prob:  1.67% Token: |75|


Ranks of the answer tokens: [(' 99', 1233)]

In [34]:
example_prompt = "1 table 2. 3 table 4. 8 table 9. 100 table"
example_answer = " 101"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' table', ' 2', '.', ' 3', ' table', ' 4', '.', ' 8', ' table', ' 9', '.', ' 100', ' table']
Tokenized answer: [' 101']


Performance on answer token:
Rank: 0        Logit: 14.75 Prob: 28.30% Token: | 101|

Top 0th token. Logit: 14.75 Prob: 28.30% Token: | 101|
Top 1th token. Logit: 14.19 Prob: 16.09% Token: | 11|
Top 2th token. Logit: 13.75 Prob: 10.32% Token: |
|
Top 3th token. Logit: 13.36 Prob:  7.01% Token: | 10|
Top 4th token. Logit: 13.16 Prob:  5.72% Token: | 1|
Top 5th token. Logit: 12.68 Prob:  3.56% Token: | 100|
Top 6th token. Logit: 12.25 Prob:  2.32% Token: | 102|
Top 7th token. Logit: 11.98 Prob:  1.77% Token: | 2|
Top 8th token. Logit: 11.66 Prob:  1.28% Token: | 111|
Top 9th token. Logit: 11.52 Prob:  1.11% Token: | 110|


Ranks of the answer tokens: [(' 101', 0)]

In [35]:
example_prompt = "January"
example_answer = " February"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'January']
Tokenized answer: [' February']


Performance on answer token:
Rank: 97       Logit:  9.28 Prob:  0.02% Token: | February|

Top 0th token. Logit: 14.59 Prob:  3.66% Token: | 28|
Top 1th token. Logit: 14.55 Prob:  3.51% Token: | 26|
Top 2th token. Logit: 14.51 Prob:  3.37% Token: | 27|
Top 3th token. Logit: 14.50 Prob:  3.33% Token: | 1|
Top 4th token. Logit: 14.48 Prob:  3.25% Token: | 13|
Top 5th token. Logit: 14.45 Prob:  3.18% Token: | 14|
Top 6th token. Logit: 14.45 Prob:  3.18% Token: | 12|
Top 7th token. Logit: 14.44 Prob:  3.13% Token: | 15|
Top 8th token. Logit: 14.40 Prob:  3.02% Token: | 7|
Top 9th token. Logit: 14.39 Prob:  2.99% Token: | 22|


Ranks of the answer tokens: [(' February', 97)]

In [36]:
example_prompt = "95 96"
example_answer = " 97"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '95', ' 96']
Tokenized answer: [' 97']


Performance on answer token:
Rank: 2        Logit: 12.01 Prob:  5.56% Token: | 97|

Top 0th token. Logit: 12.69 Prob: 11.00% Token: |
|
Top 1th token. Logit: 12.03 Prob:  5.72% Token: | 95|
Top 2th token. Logit: 12.01 Prob:  5.56% Token: | 97|
Top 3th token. Logit: 11.91 Prob:  5.06% Token: | 96|
Top 4th token. Logit: 11.51 Prob:  3.37% Token: |.|
Top 5th token. Logit: 11.01 Prob:  2.05% Token: | 93|
Top 6th token. Logit: 10.73 Prob:  1.56% Token: | 0|
Top 7th token. Logit: 10.70 Prob:  1.51% Token: | 99|
Top 8th token. Logit: 10.64 Prob:  1.42% Token: | 94|
Top 9th token. Logit: 10.50 Prob:  1.24% Token: | 1|


Ranks of the answer tokens: [(' 97', 2)]

In [37]:
example_prompt = "94 95 96"
example_answer = " 97"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '94', ' 95', ' 96']
Tokenized answer: [' 97']


Performance on answer token:
Rank: 0        Logit: 16.68 Prob: 77.60% Token: | 97|

Top 0th token. Logit: 16.68 Prob: 77.60% Token: | 97|
Top 1th token. Logit: 14.37 Prob:  7.72% Token: |
|
Top 2th token. Logit: 12.08 Prob:  0.78% Token: | 95|
Top 3th token. Logit: 11.58 Prob:  0.47% Token: | 96|
Top 4th token. Logit: 11.48 Prob:  0.43% Token: | 98|
Top 5th token. Logit: 11.44 Prob:  0.41% Token: | 1|
Top 6th token. Logit: 11.19 Prob:  0.32% Token: | 77|
Top 7th token. Logit: 11.13 Prob:  0.30% Token: | 27|
Top 8th token. Logit: 11.01 Prob:  0.27% Token: | 93|
Top 9th token. Logit: 10.80 Prob:  0.22% Token: | 87|


Ranks of the answer tokens: [(' 97', 0)]

In [38]:
example_prompt = "93 94 95 96"
example_answer = " 97"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '93', ' 94', ' 95', ' 96']
Tokenized answer: [' 97']


Performance on answer token:
Rank: 0        Logit: 19.59 Prob: 95.99% Token: | 97|

Top 0th token. Logit: 19.59 Prob: 95.99% Token: | 97|
Top 1th token. Logit: 15.69 Prob:  1.93% Token: |
|
Top 2th token. Logit: 13.21 Prob:  0.16% Token: | 98|
Top 3th token. Logit: 13.09 Prob:  0.14% Token: | 95|
Top 4th token. Logit: 12.68 Prob:  0.10% Token: | 96|
Top 5th token. Logit: 12.65 Prob:  0.09% Token: | 1|
Top 6th token. Logit: 12.25 Prob:  0.06% Token: | 27|
Top 7th token. Logit: 11.93 Prob:  0.05% Token: | 93|
Top 8th token. Logit: 11.89 Prob:  0.04% Token: | 77|
Top 9th token. Logit: 11.80 Prob:  0.04% Token: | %|


Ranks of the answer tokens: [(' 97', 0)]

In [40]:
example_prompt = "93 95 97 99"
example_answer = " 101"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '93', ' 95', ' 97', ' 99']
Tokenized answer: [' 101']


Performance on answer token:
Rank: 8        Logit: 12.92 Prob:  1.63% Token: | 101|

Top 0th token. Logit: 16.10 Prob: 39.42% Token: | 100|
Top 1th token. Logit: 14.44 Prob:  7.49% Token: |
|
Top 2th token. Logit: 14.23 Prob:  6.06% Token: | 99|
Top 3th token. Logit: 13.85 Prob:  4.18% Token: | 98|
Top 4th token. Logit: 13.79 Prob:  3.91% Token: | 0|
Top 5th token. Logit: 13.24 Prob:  2.27% Token: | 97|
Top 6th token. Logit: 13.05 Prob:  1.88% Token: | 95|
Top 7th token. Logit: 12.95 Prob:  1.70% Token: | 96|
Top 8th token. Logit: 12.92 Prob:  1.63% Token: | 101|
Top 9th token. Logit: 12.84 Prob:  1.51% Token: | 102|


Ranks of the answer tokens: [(' 101', 8)]

In [42]:
example_prompt = "89 91 93 95 97 99"
example_answer = " 101"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '89', ' 91', ' 93', ' 95', ' 97', ' 99']
Tokenized answer: [' 101']


Performance on answer token:
Rank: 4        Logit: 13.93 Prob:  2.84% Token: | 101|

Top 0th token. Logit: 16.73 Prob: 46.86% Token: | 100|
Top 1th token. Logit: 15.11 Prob:  9.26% Token: |
|
Top 2th token. Logit: 14.93 Prob:  7.76% Token: | 99|
Top 3th token. Logit: 14.21 Prob:  3.78% Token: | 98|
Top 4th token. Logit: 13.93 Prob:  2.84% Token: | 101|
Top 5th token. Logit: 13.54 Prob:  1.93% Token: | 102|
Top 6th token. Logit: 13.39 Prob:  1.67% Token: | 0|
Top 7th token. Logit: 13.33 Prob:  1.56% Token: | 97|
Top 8th token. Logit: 13.15 Prob:  1.31% Token: | 96|
Top 9th token. Logit: 12.97 Prob:  1.09% Token: | 95|


Ranks of the answer tokens: [(' 101', 4)]

In [43]:
example_prompt = "87 89 91 93 95 97 99"
example_answer = " 101"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '87', ' 89', ' 91', ' 93', ' 95', ' 97', ' 99']
Tokenized answer: [' 101']


Performance on answer token:
Rank: 4        Logit: 13.80 Prob:  2.96% Token: | 101|

Top 0th token. Logit: 16.63 Prob: 50.04% Token: | 100|
Top 1th token. Logit: 14.86 Prob:  8.47% Token: |
|
Top 2th token. Logit: 14.67 Prob:  7.04% Token: | 99|
Top 3th token. Logit: 13.97 Prob:  3.49% Token: | 98|
Top 4th token. Logit: 13.80 Prob:  2.96% Token: | 101|
Top 5th token. Logit: 13.51 Prob:  2.21% Token: | 102|
Top 6th token. Logit: 13.11 Prob:  1.47% Token: | 97|
Top 7th token. Logit: 12.97 Prob:  1.29% Token: | 0|
Top 8th token. Logit: 12.69 Prob:  0.98% Token: | 10|
Top 9th token. Logit: 12.67 Prob:  0.95% Token: | 103|


Ranks of the answer tokens: [(' 101', 4)]

In [45]:
example_prompt = "84 86 88 90 92 94"
example_answer = " 96"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '84', ' 86', ' 88', ' 90', ' 92', ' 94']
Tokenized answer: [' 96']


Performance on answer token:
Rank: 1        Logit: 16.99 Prob: 27.83% Token: | 96|

Top 0th token. Logit: 17.31 Prob: 38.58% Token: | 95|
Top 1th token. Logit: 16.99 Prob: 27.83% Token: | 96|
Top 2th token. Logit: 15.81 Prob:  8.53% Token: | 97|
Top 3th token. Logit: 15.43 Prob:  5.83% Token: | 94|
Top 4th token. Logit: 14.50 Prob:  2.32% Token: | 98|
Top 5th token. Logit: 14.42 Prob:  2.13% Token: |
|
Top 6th token. Logit: 14.39 Prob:  2.07% Token: | 93|
Top 7th token. Logit: 13.76 Prob:  1.11% Token: | 92|
Top 8th token. Logit: 13.61 Prob:  0.95% Token: | 100|
Top 9th token. Logit: 13.55 Prob:  0.89% Token: | 99|


Ranks of the answer tokens: [(' 96', 1)]